# 06. RAG_DocumentGPT

## RAG (Retrieval Augmented Generation, 검색 증강 생성)

- model들은 많은 데이터를 통해 학습하지만 DB나 문서와 같은 private data에 접근할 수는 없다.
- model은 이러한 data를 알 수 없으므로, 우리는 질문할 때 그와 관련있는 문서들도 같이 준비하게 된다. (vector나 저장기 활용은 뒤에서 배운다.)

### What is 'foo'?

- 우리의 model은 'foo'라는 단어를 아예 모르는 상태이다.
- 따라서 우리는 'foo'와 관련된 문서들을 준비해 가져올 것이다.
- **이 문서들을 context로써 질문과 함께 묶어 prompt를 구성해, Language Model에 보낼 것이다.**
- Model은 기존에 학습된 data와 추가로 전달 받은 context로써의 data를 모두 갖게 된다.
- 이러한 개념을 ***RAG*** 이라고 한다.

## Retrieval
- Langchain의 모듈
- Source -> Load -> Transform -> Embed -> Store -> Retrieve
  
### UnstructedFileLoader
- 대부분 파일들에 호환되는 범용 loader

### 문서 분할

- 예제의 경우 전체 챕터가 하나의 문서에 들어가 있다. 
- 너무 큰 한 덩어리 데이터를 분할하여 파일의 필요한 부분만을 사용하고자 분할한다.
- `chunk_size`를 지정해 잘라낸 각 덩어리는 작아졌지만 문단까지 잘려 그냥 사용하기엔 문제가 많았음.
- `chunk_overlap` 을 지정해 앞 chunk의 일부를 뒤 chunk의 시작으로 끌고올 수 있다. 단, 이로 인해 chunk 간 중복되는 부분이 생긴다.
- 

### Token

위에서 text를 chunk 단위로 분류했지만, 엄밀히 token과 letter는 다른 것이다.
- token은 여러 개의 문자나 단어가 될 수도 있다.

문장을 Token으로 분리했을 때, Model이 받게 되는 데이터는 각 Token에 대한 ID값이다.
- nomad : nom / ad -> [17101, 329] 와 같은 식
- 이를 기준으로 AI는 'token_17101 다음에 token_329가 올 확률이 높다' 등의 예측 가능

### Tiktoken

이러한 데이터의 Tokenizing을 위해 사용하는 라이브러리

## Embedding

지금까지 위의 RAG 중 Load-Transform 단계까지는 수행했다.   
이제 컴퓨터가 이해할 수 있도록 data를 Embedding해 주어야 한다.

문서의 split 처리된 data마다 각각의 (3D)vector를 만들 것이며, 이를 위해 embedding model을 사용한다.

Masculinity | Femininity | Royalty 를 우리의 3개 차원으로 둔다고 가정해 보자.
여러 단어들을 아래와 같이 우리의 차원에서 정의해 보자.

[word(Masculinity, Femininity, Royalty)]
- King  (0.9, 0.1, 1.0)
- Queen (0.1, 0.9, 1.0)
- Man   (0.9, 0.1, 0,0)

'우리의 차원에서 King은 0.9의 남성성, 1.0의 충성심, 0.1의 여성성을 가진다' 라는 것을 3차원 vector로 나타내었다.   

- Vector Calculus
King-Man = (0, 0, 1.0) = ***Royal***   
이는 'King과 Man은 우리의 차원에서 Royalty=1.0의 차이를 갖는다.' 로 해석할 수 있으며,   
우리의 차원에서 이러한 단어는 'royal'과 같이 정의내릴 수 있다.   

Man+Royal = (0.9, 0.1, 1.0) = ***King***   
이처럼 단어 간의 연관성(가깝고 멂)을 벡터 연산을 통해 나타낼 수 있다.   
(+) : 벡터 간 연관성은 벡터 간 거리로 판단한다.   

이를 통해 특정 단어에 대해 연관성이 높은 단어를 **Search**할 수 있게 된다.   
(+) 물론 연관성의 range에 대한 것은 개발자가 정해야 할 것 같다.   

많은 추천 알고리즘은 이와 같은 방식으로 동작한다.   


차원의 개수가 늘어날수록 한 단어가 담고 있는(표현할 수 있는) 속성의 개수가 많을 것이고,   
이를 통해 비슷한 영역에 있는 단어나 문서를 찾을 수 있다.   
(참고 : The conspiracy to make AI seem harder than it is! By Gustav Soderstrom (Spotify R&D) (Youtube))


## Vector Store

일종의 DB로 생각할 수 있다. Embed를 통해 vector를 만들고 이곳에 저장한 후,   
vector store에서 검색할 수 있게 된다. (관련있는 문서만 찾아낼 수 있게 된다.)

매 코드 실행마다 문서를 embed하기 보다는, embed한 내용을 저장하여 사용하자.   
Langchain에서는 embed 결과를 cache에 저장할 수 있도록 기능을 제공해 준다.

우리는 Chroma를 사용할 것이다. Local에서 실행될 것임.   

vectorstore로 보내지는 문서의 크기에 따라 비용이 달라지므로,   
의미가 왜곡되지 않는 선에서 적정한 split chunksize를 정하는 것이 중요할 것으로 보임.


In [4]:
import os
os.environ["TIKTOKEN_CACHE_DIR"]="./etc"

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryMemory, ConversationSummaryBufferMemory

from langchain.callbacks import StreamingStdOutCallbackHandler

## import tiktoken_ext.openai_public


## Document 분할
##from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.memory import ConversationSummaryMemory, ConversationSummaryBufferMemory
from langchain.vectorstores import Chroma

from langchain.storage import LocalFileStore

## loader import
from langchain.document_loaders import UnstructuredFileLoader ##TextLoader, PyPDFLoader 등 범용

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,     ##Streaming 옵션 ON
    callbacks=[StreamingStdOutCallbackHandler()]
)




SSLError: HTTPSConnectionPool(host='openaipublic.blob.core.windows.net', port=443): Max retries exceeded with url: /gpt-2/encodings/main/vocab.bpe (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))

In [ ]:
import os
os.environ["TIKTOKEN_CACHE_DIR"]="./etc"

splitterTiktoken = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",     ## CharacterTextSplitter에 한해 사용
    chunk_size=600,     ## split size
    chunk_overlap=100,  ## split 시 앞 문단의 끝 일부를 같이 가져옴   
)

In [9]:


# validate
##assert os.path.exists(os.path.join(tiktoken_cache_dir,"9b5ad71b2ce5302211f9c61530b329a4922fc6a4"))

In [2]:



## embedded data의 cache 저장을 위한 storage 생성 (local)
cache_dir = LocalFileStore("./.cache/")

## 파일 분할 Splitter (문장 끝이나 문단 끝부분마다 잘라줌)
"""
splitter = CharacterTextSplitter(
    separator="",     ## CharacterTextSplitter에 한해 사용
    chunk_size=600,     ## split size
    chunk_overlap=100,  ## split 시 앞 문단의 끝 일부를 같이 가져옴
    ##참고 : len은 기본 라이브러리 함수임
)
"""


'\nsplitter = CharacterTextSplitter(\n    separator="",     ## CharacterTextSplitter에 한해 사용\n    chunk_size=600,     ## split size\n    chunk_overlap=100,  ## split 시 앞 문단의 끝 일부를 같이 가져옴\n    ##참고 : len은 기본 라이브러리 함수임\n)\n'

In [3]:


##txt loader
loader = UnstructuredFileLoader("./files/document.txt")
##loader.load()

SSLError: HTTPSConnectionPool(host='openaipublic.blob.core.windows.net', port=443): Max retries exceeded with url: /gpt-2/encodings/main/vocab.bpe (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))

In [7]:
embedder = OpenAIEmbeddings()

##vector = embedder.embed_query("Hi")
## 아래의 output은 단어 'Hi'의 vector값이다.

## 총 1536개의 차원으로 단어를 표현한다는 의미이다.
## len(vector)

docs = loader.load_and_split(text_splitter=splitterTiktoken)
embeddings = OpenAIEmbeddings()

## Cache Embedding (CacheBackedEmbedding는 Embedder를 받아야 함

cached_embeddings = CacheBackedEmbeddings(embeddings, cache_dir)

## chche hit -> load from cache_dir, cache miss -> embed on embeddings
vectorstore = Chroma.from_documents(docs, cached_embeddings)

NameError: name 'loader' is not defined

In [ ]:
## vectorstore에서 유사도 검색을 수행 (similarity_search)
## 우리 문서의 단어를 vector로 변환해 공간을 구성하고, 이 공간에서 단어 간의 연관성을 찾음
